In [17]:
import numpy as np
import cv2 as cv2
import scipy.misc
from scipy.ndimage.filters import gaussian_filter

path = "/Users/jackychencw/Desktop/CSC420/A1/"

In [18]:
def readImg(filename):
    image = cv2.imread(path + filename, 0)
    return image

In [19]:
# def writeFile(filename):
#     f= open(path + filename,"w+")

In [32]:
## I: an input grayscale image
## h: a filter
## mode: a string, valid, same or full
## outputs a results J

def MyCorrelation(I, h, mode):
    img = readImg(I)
    img_x = img.shape[1]
    img_y = img.shape[0]
    h_x = h.shape[1]
    h_y = h.shape[0]
    l = h_x//2
    f = h.flatten('F')
    
    if mode == "full":
        new_x = img_x + (h_x - 1) * 2
        new_y = img_y + (h_y - 1) * 2
        
        new_img = np.zeros((new_y, new_x))
        
        padding_img = np.pad(img, ((h_y - 1, h_y - 1), (h_x - 1, h_x - 1)), "constant")
        
        for i in range(l, new_y - l):
            for j in range(l, new_x - l):
                t = padding_img[i - l: i + l + 1, j - l : j + l + 1]
                t = t.flatten('F')
                result = np.dot(f.T, t)
                new_img[i,j] = result
                
    elif mode == "same":
        new_img = np.zeros(img.shape)
        half_y = h_y//2
        half_x = h_x//2
        padding_img = np.pad(img, ((half_y, half_y),(half_x, half_x)), "constant")
        
        for i in range(l, img_y - l + 1):
            for j in range(l, img_x - l + 1):
                t = padding_img[i - l: i + l + 1, j - l : j + l + 1]
                t = t.flatten('F')
                result = np.dot(f.T, t)
                new_img[i,j] = result
                
    elif mode == "valid":
        new_x = img_x - h_x + 1
        new_y = img_y - h_y + 1
        new_img = np.zeros((new_y, new_x))
        
        for i in range(l, new_y - l + 1):
            for j in range(l, new_x - l + 1):
                t = img[i - l: i + l + 1, j - l : j + l + 1]
                t = t.flatten('F')
                result = np.dot(f.T, t)
                new_img[i,j] = result
    return new_img

In [22]:
h = np.ones((7,7))
valid = MyCorrelation("gray.jpg", h, "valid")
scipy.misc.imsave(path + 'cor_valid.jpg', valid)
same = MyCorrelation("gray.jpg", h, "same")
scipy.misc.imsave(path + 'cor_same.jpg', same)
full = MyCorrelation("gray.jpg", h, "full")
scipy.misc.imsave(path + 'cor_full.jpg', full)

In [33]:
h = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
full = MyCorrelation("color_noised.jpg", h, "full")
scipy.misc.imsave(path + 'color_edge.jpg', full)

In [23]:
def MyConvolution(I, h, mode):
    img = readImg(I)
    img_x = img.shape[1]
    img_y = img.shape[0]
    h_x = h.shape[1]
    h_y = h.shape[0]
    l = h_x//2
    h = np.flip(np.flip(h, 0),1)
    f = h.flatten('F')
    
    if mode == "full":
        new_x = img_x + (h_x - 1) * 2
        new_y = img_y + (h_y - 1) * 2
        
        new_img = np.zeros((new_y, new_x))
        
        padding_img = np.pad(img, ((h_y - 1, h_y - 1), (h_x - 1, h_x - 1)), "constant")
        
        for i in range(l, new_y - l):
            for j in range(l, new_x - l):
                t = padding_img[i - l: i + l + 1, j - l : j + l + 1]
                t = t.flatten('F')
                result = np.dot(f.T, t)
                new_img[i,j] = result
                
    elif mode == "same":
        new_img = np.zeros(img.shape)
        half_y = h_y//2
        half_x = h_x//2
        padding_img = np.pad(img, ((half_y, half_y),(half_x, half_x)), "constant")
        
        for i in range(l, img_y - l + 1):
            for j in range(l, img_x - l + 1):
                t = padding_img[i - l: i + l + 1, j - l : j + l + 1]
                t = t.flatten('F')
                result = np.dot(f.T, t)
                new_img[i,j] = result
                
    elif mode == "valid":
        new_x = img_x - h_x + 1
        new_y = img_y - h_y + 1
        new_img = np.zeros((new_y, new_x))
        
        for i in range(l, new_y - l + 1):
            for j in range(l, new_x - l + 1):
                t = img[i - l: i + l + 1, j - l : j + l + 1]
                t = t.flatten('F')
                result = np.dot(f.T, t)
                new_img[i,j] = result
    return new_img

In [26]:
h = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])
new_img = MyConvolution("gray.jpg", h, "valid")
scipy.misc.imsave(path + 'con2.jpg', new_img)
valid = MyCorrelation("gray.jpg", h, "valid")
scipy.misc.imsave(path + 'valid.jpg', valid)

In [14]:
def portrait(I, h, mode, mask):
    img = readImg(I)
    new_img =gaussian_filter(img, sigma=7)
    result = np.zeros(new_img.shape)
    where_0 = np.where(mask == 0)
    where_1 = np.where(mask == 1)
    result[where_0] = new_img[where_0]
    result[where_1] = img[where_1]
    return result

In [16]:
mask = np.zeros((200,200))
h = np.ones((3,3))
mask = np.pad(mask, ((300,300),(300,300)), "constant")
img = portrait("gray.jpg", h, "same", mask)
scipy.misc.imsave(path + 'portrait2.jpg', img)

In [31]:
f = np.array([[1,1,1],[1,0,1],[1,1,1]])
denoise = MyCorrelation("salt-and-pepper.jpg", f, "valid")
scipy.misc.imsave(path + 'sap-box3.jpg', denoise)